In [1]:
import Base.copy
using Distributions
include("Mondrian_Forest_Classifier.jl"); 
include("Mondrian_extention_utils.jl");
include("Mondrian_extention.jl")

Extend_Mondrian_Block! (generic function with 1 method)

In [2]:
using MLBase
function Fakedata(n,dim) 
    x = randn(n,dim)
    y = (sum(exp.(x)/(1+exp.(x)),2)).>0.5
    return 1.0*x,1*y[:,1]
end

d=5
n=1000
X, Y = Fakedata(n,d);
Y = Y.+1;

n=1
x,y = Fakedata(n,d)
y = y.+1;

In [3]:
D=[X,Y]
d_new = [x,y] 
T = Mondrian_Tree()

Mondrian_Tree(Nullable{Mondrian_Node}(), Mondrian_Node[])

In [4]:
T_classic = Sample_Mondrian_Tree!(T,1e9,D[1],D[2]);  #compute base tree

In [5]:
T_online = Extend_Mondrian_Tree!(T_classic,1e9,d_new);  #extent base tree

### Problem: The Mondrian_Tree function does not return, tree can therefore not be used for extension!!

In [6]:
# create new dataset including previous training data + new datapoint
All_data = zeros(1001,5)
All_data[1:1000,1:5]=X[1:1000,1:5]
All_data[1001,1:5]=x[1,1:5]

all_labels = Array{Int64}(1001,1)
all_labels[1:1000,:]=Y
all_labels[1001,:]=y[1,1]

1

In [7]:
T_full = Sample_Mondrian_Tree!(T,1e9,All_data,all_labels);  #compute full tree of the whole dataset

### Compare prediction accuracy

In [8]:
compute_predictive_posterior_distribution!(T_full,10*size(All_data,2))

In [9]:
pred = []
    for i in 1:size(All_data,1)
        p = predict!(T_full,All_data[i,:],10*size(All_data,2))
        if p[1] > p[2]
            push!(pred,1)
        else
            push!(pred,2)
        end
    end


In [10]:
counter = 0
for i=1:1001
    if pred[i]==all_labels[i]
        counter = counter+1
    end
end
counter = counter /1001

0.5584415584415584

In [11]:
compute_predictive_posterior_distribution!(T_online,10*size(All_data,2))

In [12]:
 pred = []
        for i in 1:size(All_data,1)
            p = predict!(T_online,All_data[i,:],10*size(All_data,2))
            if p[1] > p[2]
                push!(pred,1)
            else
                push!(pred,2)
            end
        end

In [13]:
counter = 0
for i=1:1001
    if pred[i]==all_labels[i]
        counter = counter+1
    end
end
counter = counter /1001

0.5584415584415584

## Conclusion: get the same accuracy weather we incorperate the datapoint later or immediatly

## To Do:

Clean up utils files, test on more classes, test on extending to more datapoints successively, implement paused mondrians 